# Зямзина Анастасия Сергеевна 3530903/70302   2 часть #
## 8 вариант ##
### Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных ###

In [419]:
import numpy as np
import pandas as pd
import matplotlib
import math as m

In [420]:
variant = 7

In [421]:
data = pd.read_csv('data.csv')
#удаляем данные 1 столбца, так как они в принципе не очень то и нужны, ну правда
#data = data.drop(data.columns[0],axis = 1)
contday = pd.read_csv("context_day.csv")
contday = contday.drop(contday.columns[0],axis = 1)
contpl = pd.read_csv("context_place.csv")
contpl = contpl.drop(contpl.columns[0],axis = 1)
print("Исходные данные\n",data,"\n",contday,"\n", contpl)

Исходные данные
    Unnamed: 0   Movie 1   Movie 2   Movie 3   Movie 4   Movie 5   Movie 6  \
0      User 1        -1         4         1         5        -1        -1   
1      User 2         4         5         2         3         3         4   
2      User 3         4         2         5         4         1         5   
3      User 4         5        -1         2        -1        -1         2   
4      User 5         4         3         3         3         5         3   
5      User 6         2        -1         3         3         3         3   
6      User 7         4         1         4         4         1         4   
7      User 8        -1         5         5         3         3         3   
8      User 9         1         2         3         1         5         3   
9     User 10         5        -1         1        -1         3         3   
10    User 11        -1         5        -1         1         5         2   
11    User 12         5         4         4        -1      

## Расчет оценки фильма будет проводится чреди фильмов, которые пользователь еще не посмотрел ##

In [422]:
duser = data.iloc[variant].copy()
duser = duser.transpose()
for i in range(0,31):
    if duser.iloc[i] != -1:
        duser.iloc[i] = np.NaN
answer = duser.dropna()
a1 = answer.keys()
answer

 Movie 1     -1
 Movie 11    -1
 Movie 13    -1
 Movie 18    -1
 Movie 21    -1
 Movie 28    -1
Name: 7, dtype: object

## Оценка будет составляться из относительности просмотра фильмов по выходным к общему количеству просмотров (чем больше фильм смотрели, тем более по выходным, тем выше оценка) ##

In [423]:
c = 0
contday = contday.replace({' -1':np.NaN})
film_vac_ev = answer.copy()
for i in range(len(a1)):
    film = a1[i]
    c = contday[a1[i]].value_counts() 
    sSat = c[[' Sun']]
    sSun = c[[' Sat']]
    c = c.dropna()
    views = c.sum()
    # основная формула расчта: отношение суммы просмотров на выходных к общему количеству просмотров ( вычитается из 1, т.к. чем
    # выше просмотры => тем меньше дробь, а я в дальнейшем планирую суммировать оценки => мне нужен не меньший, а больший рейтинг)
    sumVac = 1 - round((sSat[0] + sSun[0]) / views,3)
    film_vac_ev.iloc[i] = sumVac 
        
film_vac_ev

 Movie 1     0.656
 Movie 11      0.7
 Movie 13    0.667
 Movie 18    0.645
 Movie 21    0.774
 Movie 28    0.706
Name: 7, dtype: object

**Просмотры дома**

In [424]:
c = 0
film_home_ev = answer.copy()
contpl = contpl.replace({' -1':np.NaN})
for i in range(len(a1)):
    film = a1[i]
    c = contpl[a1[i]].value_counts() 
    sh = c[[' h']]
    c = c.dropna()
    views = c.sum()
    sumh = 1 - round(sh[0] / views,3)
    film_home_ev.iloc[i] = sumh
film_home_ev   

 Movie 1      0.75
 Movie 11    0.733
 Movie 13    0.667
 Movie 18     0.71
 Movie 21    0.742
 Movie 28    0.706
Name: 7, dtype: object

## Возьмем данные из предыдущего задания. Общую оценку фильма просуммируем с оценками о выходных и домашнем просмотре и получим новый рейтинг ##

In [425]:
import json

with open('res.json') as f:
    d = json.load(f)
    print(d)

datafilms = pd.json_normalize(d)
datafilms = datafilms.transpose()
datafilms.rename(index={'0':'1'})

{' Movie 1': 1.992, ' Movie 11': 1.467, ' Movie 13': 1.99, ' Movie 18': 2.481, ' Movie 21': 1.225, ' Movie 28': 1.733}


,0
Movie 1,1.992
Movie 11,1.467
Movie 13,1.990
Movie 18,2.481
Movie 21,1.225
Movie 28,1.733


In [434]:
n_rate = answer.copy()
for i in range(len(n_rate)):
    n_rate[i] = film_vac_ev[i] + datafilms.iloc[[i]] + film_home_ev[i] 

for i in range(len(n_rate)):
    print("Расчитанный рейтинг:",n_rate[i][0])
    
k = n_rate[0].max()
print("Максимальный рейтинг",k.max())

Расчитанный рейтинг:  Movie 1    3.398
Name: 0, dtype: float64
Расчитанный рейтинг:  Movie 11    2.9
Name: 0, dtype: float64
Расчитанный рейтинг:  Movie 13    3.324
Name: 0, dtype: float64
Расчитанный рейтинг:  Movie 18    3.836
Name: 0, dtype: float64
Расчитанный рейтинг:  Movie 21    2.741
Name: 0, dtype: float64
Расчитанный рейтинг:  Movie 28    3.145
Name: 0, dtype: float64
Максимальный рейтинг 3.398


## Ответ ##

In [438]:
ind = np.argpartition(n_rate[0], -1)[-1:]
movietoreccomend = ind.drop(ind.columns[0], axis = 1)
movietoreccomend[2] = k.max()
print("Рекомендуемый фильм:",movietoreccomend[2])


Рекомендуемый фильм:  Movie 1    3.398
Name: 2, dtype: float64


## Подгтовка данных для json ##

In [439]:
import json
js2 = movietoreccomend.to_json(orient='columns')
df = datafilms.rename(columns={0:1})
js1 = df.to_json(orient='columns')
var = {'user':variant}
var = json.loads(r)
result1 = json.loads(js1)
result2 = json.loads(js2)
with open('zyamzina_results.json', 'w') as fl:
    fl.write(json.dumps(var, indent=4))
    fl.write(json.dumps(result1, indent=4))
    fl.write(json.dumps(result2, indent=4))